In [2]:
from pymtl3 import *
from pymtl3.passes.backends.verilog import *

In [3]:
# Simulation
class Adder(Component):
    def construct (s):
        s.a = InPort(32)
        s.b = InPort(32)
        s.q = OutPort(32)

        @update
        def comb():
            s.q @= s.a + s.b

In [4]:
a = Adder()
a.apply(DefaultPassGroup(linetrace=True, textwave=True))
a.sim_reset()

a.a @= 5
a.b @= 7
a.sim_eval_combinational()
print(a.q.uint())

12


In [5]:
a.sim_tick()
a.print_textwave()


      |0    |1    |2    |3    

  clk /‾‾\__/‾‾\__/‾‾\__/‾‾\__

reset ‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾\_____

    a  0000000          |+0005

    b  0000000          |+0007

    q  0000000          |+000c



In [6]:
from src.cl.decoder import Decode, SingleInstDecode
from src.cl.register_rename import RegisterRename
from src.cl.dispatch import Dispatch
from src.cl.reorder_buffer import ReorderBuffer
from pymtl3.passes.backends.verilog import VerilogTranslationPass

model = Decode()
model.set_metadata(VerilogTranslationPass.enable, True)
model.set_metadata(VerilogTranslationPass.explicit_file_name, "./translated/" + model.__module__.split(".")[-1])
model.set_metadata(VerilogTranslationPass.explicit_module_name, model.__class__.__name__)

# Generate Verilog
model.apply( VerilogTranslationPass() )

PyMTLSyntaxError: 
In file /Users/curtisbucher/Desktop/ramp-core/src/cl/register_rename.py, Line 77, Col 19:
  if s.free_list[i] and s.pdst1 == 0:
     ^
- Boolean operations are not translatable due to inconsistent semantics between Python and PyMTL. Please consider using bitwise &, |, and ~ instead!

In [7]:
from src.fl.util import cascading_priority_encoder

for x in range(2**32):
    free_list = Bits64(x) << 1
    a, b = cascading_priority_encoder(2, free_list)

    pdst1, pdst2 = 0,0#Wire(Bits32), Wire(Bits32)
    for i in range(64):
        if free_list[i] and pdst1 == 0:
            pdst1 = i
        elif free_list[i] and pdst1 != 0:
            pdst2 = i
            break

    if a != pdst1 or b != pdst2:
        print(f"fl: {free_list.bin()} \na: {a} pdst1: {pdst1} \nb: {b}  pdst2: {pdst2}")
        # break